In [ ]:
import os
import numpy as np
import pandas as pd

from collections import defaultdict
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [ ]:
print(os.path.abspath('.'))
# tflog_dir must be the directory immediately above the runs and each run must have the same shape. No aggregation of multiple subdirectories for now.
tflog_dir = './fpsb/asymmetric/uniform/2p' # where to find the runs to aggregate
output_dir = tflog_dir                 # where to writethe output files

In [ ]:
# based on https://stackoverflow.com/a/57411105/4755970
def tabulate_events(dpath):

    final_out = {}
    
    # runs are all subdirectories that don't start with '.' (exclude '.ipython_checkpoints')
    # add more filters as needed
    runs = [x.name for x in os.scandir(dpath) if x.is_dir() and not x.name.startswith('.')]
    
    for dname in runs:
        print(f"Converting run {dname}",end="")
        ea = EventAccumulator(os.path.join(dpath, dname)).Reload()
        tags = ea.Tags()['scalars']

        out = {}

        for tag in tags:
            tag_values=[]
            wall_time=[]
            steps=[]

            for event in ea.Scalars(tag):
                tag_values.append(event.value)
                wall_time.append(event.wall_time)
                steps.append(event.step)

            out[tag]=pd.DataFrame(data=dict(zip(steps,np.array([tag_values,wall_time]).transpose())), columns=steps,index=['value','wall_time'])

        if len(tags)>0:      
            df= pd.concat(out.values(),keys=out.keys())
            df.to_csv(os.path.join(output_dir, f'result_{dname}.csv'))
            print("- Done")
        else:
            print('- Not scalers to write')
            
        final_out[dname] = df
        
    return final_out

In [ ]:
if __name__ == '__main__':
    steps = tabulate_events(tflog_dir)
    pd.concat(steps.values(),keys=steps.keys()).to_csv(os.path.join(output_dir, 'all_results.csv'))